## Using the FDIC API, let's look at total assets and liabilities for Silicon Valley Bank over time. The data shows that Silicon Valley Bank saw lots of growth over time in its total assets and liabilities. It looks like total assets were always higher than total liabilities for all reported quarters. 

In [1]:
"""
Use the FDIC API to get total assets and liabilities for Silicon Valley Bank and compare the two
"""
import requests
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go


def get_financials_data():
    """Get financials data"""
    bank = "SILICON VALLEY BANK"
    fields = "NAME,STNAME,REPYEAR,RISDATE,REPDTE"
    # add fields about assets
    fields += ",ASSET"
    # add fields about liabilities
    fields += ",DEP,LIAB"
    # add any other fields
    fields += ""
    limit = 200

    root = "https://banks.data.fdic.gov/api/financials"
    url = f'{root}?filters=NAME:"{bank}"&fields={fields}&limit={limit}'

    response = requests.get(url)
    results = response.json()
    results = [x["data"] for x in results["data"]]
    df = pd.DataFrame.from_records(results)
    df["REPDTE"] = pd.to_datetime(df["REPDTE"])
    df["RISDATE"] = pd.to_datetime(df["RISDATE"])
    df["REPYEAR"] = pd.to_datetime(df["REPYEAR"])
    df["DIFF"] = df["ASSET"] - df["LIAB"]
    return df

df = get_financials_data()

fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
fig.add_trace(
    go.Scatter(
        x=df["REPDTE"],
        y=df["ASSET"],
        mode="lines+markers",
        name="Total assets",
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=df["REPDTE"],
        y=df["LIAB"],
        mode="lines+markers",
        name="Total liabilities",
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Bar(
        x=df["REPDTE"],
        y=(df["LIAB"] / df["ASSET"]) * 100,
        name="Liabilities/Assets",
    ),
    row=2,
    col=1,
)
fig.update_layout(
    yaxis=dict(title="Dollars $", type="log"), yaxis2=dict(title="Liabilities/Assets %"), height=1000
)
fig.show()

In [2]:
import pandas as pd
from alpha_vantage.fundamentaldata import FundamentalData
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import time
import os

In [3]:
api_key = os.environ['alpha_vantage_api_key']

In [4]:
# Initialize the API client
fd = FundamentalData(api_key)

In [5]:
def convert_dtypes(df):
    def contains_dates(series):
        date_count = 0
        for value in series:
            try:
                pd.to_datetime(value, errors='raise')
                date_count += 1
            except:
                pass
            if date_count >= 2:
                return True
        return False

    def contains_only_letters(series):
        for value in series:
            if not isinstance(value, str) or not value.isalpha():
                return False
        return True

    for column in df.columns:
        # If the column is an object dtype and contains only letters, skip the conversion
        if df[column].dtype == 'object' and contains_only_letters(df[column]):
            continue

        # If the column is an object dtype, try converting it to a numeric dtype
        if df[column].dtype == 'object':
            numeric_column = pd.to_numeric(df[column], errors='ignore')
            
            # Only convert to datetime if numeric conversion is not possible and the column contains dates
            if numeric_column.dtype == 'object' and contains_dates(df[column]):
                df[column] = pd.to_datetime(df[column], errors='ignore')
            else:
                df[column] = numeric_column

    return df

# Function to fetch financial data
def fetch_financial_data(function, symbol):
    data, metadata = getattr(fd, function)(symbol=symbol)
    df = pd.DataFrame(data).T
    return df

def get_balance_sheet_and_car(bank, api_key):
    fd = FundamentalData(key=api_key)
    balance_sheet = fetch_financial_data("get_balance_sheet_quarterly", bank).T
    convert_dtypes(balance_sheet)
    balance_sheet['CAR'] = balance_sheet['totalShareholderEquity'] / balance_sheet['totalAssets']
    time.sleep(15)
    
    return balance_sheet

In [6]:
bank_symbols = [
    'SIVB', 
    'PACW', 
    'SBNY', 
    # 'FRC', 
    'RY', 
    'EWBC'
]

In [7]:
balance_sheet_dict = {}
for bank in bank_symbols:
    balance_sheet = get_balance_sheet_and_car(bank, api_key)
    balance_sheet_dict[bank] = balance_sheet

In [8]:
bank_names = {
    'SIVB': 'SVB Financial',
    'PACW': 'PacWest Bancorp',
    'SBNY': 'Signature Bank',
    'FRC': 'First Republic Bank',
    'RY': 'Royal Bank of Canada',
    'EWBC': 'East West Bancorp'
}

## Let's explore the relationship between total assets and liabilities more by looking at the Capital Adequacy Ratio (CAR) of Silicon Valley Bank and other institutions. 

### Bing Chat explains CAR as:

### Capital Adequacy Ratio (CAR) is a measure of a bank’s available capital expressed as a percentage of its risk-weighted credit exposures. It is used to protect depositors and promote the stability and efficiency of financial systems around the world.

### he formula for CAR is:

### CAR = (Tier 1 Capital + Tier 2 Capital) / Risk-Weighted Assets

### Tier 1 capital consists of common stock, disclosed reserves, and retained earnings. Tier 2 capital consists of undisclosed reserves and subordinated debt.

### Unfortunately, what we have plotted here is a poor man's version of the above. It is:

### (Total assets - Total liabilities) / Total assets

In [9]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

for bank in bank_symbols:
    balance_sheet = balance_sheet_dict[bank]
    fig.add_trace(go.Scatter(x=balance_sheet['fiscalDateEnding'], y=balance_sheet['CAR'], name=bank_names[bank]))

fig.update_layout(
    title="Capital Adequacy Ratio (CAR) Over Time",
    xaxis_title="Fiscal Date Ending",
    yaxis_title="Capital Adequacy Ratio (CAR)"
)

fig.show()

### Finally, let's look at historic failure information for banks. Bank failures happen in clusters around big economic events, such as shortly after the great depression, 1980s savings and loans crisis, and the recent 2008 financial crisis. There is also a background level of bank failures. The median number of bank failures per quarter is around 2. 

In [10]:
"""
Use the FDIC API to get bank failure information over the years
"""
import plotly.express as px

def get_failure_data():
    """Get failure data"""
    fromyr = 1900
    toyr = 2023
    fields = "NAME,CERT,FIN,CITYST,FAILYR,FAILDATE,SAVR,RESTYPE,RESTYPE1,QBFDEP,QBFASSET,COST"
    limit = 10_000

    root = "https://banks.data.fdic.gov/api/failures"
    url = f'{root}?filters=FAILYR:["{fromyr}" TO "{toyr}"]&fields={fields}'
    url += "&sort_by=FAILDATE&sort_order=DESC"
    url += f"&limit={limit}&offset=0"

    response = requests.get(url)
    results = response.json()
    results = [x["data"] for x in results["data"]]
    df = pd.DataFrame.from_records(results)
    return df.sort_values("FAILYR")

df = get_failure_data()

# do some grouping
df["FAILDATE"] = pd.to_datetime(df["FAILDATE"])
grouped = df.groupby(pd.Grouper(key="FAILDATE", axis=0, freq="3M"))
monthly_counts = grouped.size()
monthly_counts.name ="Counts"
print(monthly_counts)
print("Statistics records began")
print(monthly_counts.describe())
recent_counts = monthly_counts["2000-01-01":]
print("Statistics since 2000")
print(recent_counts.describe())

# plot
fig = px.bar(monthly_counts)
fig.update_layout(height=1_000)
fig.show()

FAILDATE
1934-04-30    1
1934-07-31    3
1934-10-31    4
1935-01-31    3
1935-04-30    4
             ..
2019-10-31    2
2020-01-31    1
2020-04-30    2
2020-07-31    0
2020-10-31    2
Freq: 3M, Name: Counts, Length: 347, dtype: int64
Statistics records began
count    347.000000
mean      11.827089
std       26.512819
min        0.000000
25%        1.000000
50%        2.000000
75%        8.000000
max      282.000000
Name: Counts, dtype: float64
Statistics since 2000
count    84.000000
mean      6.940476
std      11.936934
min       0.000000
25%       1.000000
50%       2.000000
75%       6.000000
max      49.000000
Name: Counts, dtype: float64
